# Development 


## <span style="color:red">Name: Primary Task</span>   
**Primary Task:** One of *Image to Value(s)*, *Image to Image*, or *2D to 3D*  </span>  

### <span style="color:red">Name: Primary Focus</span>      
<span style="color:red">**Primary Focus:** Describe the main goal of your use case, such as *segmentation* or *object counting*. </span>  

### <span style="color:red">Name: Application</span>  
<span style="color:red">**Application:** Specify the implemented exemplary application for your use case, e.g., *segmentation of cellular structures*.  </span>  

#### <span style="color:red">Name: Challenge</span>    
<span style="color:red">**Challenge:** If your use case addresses a specific challenge (e.g., explainability, handling small datasets), mention it here. </span>   

#### <span style="color:red">Required Labels</span>  
<span style="color:red">**Required Labels:** List the necessary labels for training your model (e.g., *segmentation mask*, *location*).  </span>  

---  

## TL;DR 🧬✨
<span style="color:red">Add a short description of your use case here.</span>    

You can also include a teaser image like this:  
`![Teaser](./images/Teaser.png)`  




# Setup and Imports

In [1]:
# imports from the template 
from deepEM.Utils import create_text_widget, print_info, find_file
from deepEM.Logger import Logger
from deepEM.ModelTuner import ModelTuner

# costum implementation
from src.ModelTrainer import ModelTrainer


# import all required libraries
from pathlib import Path 
import os



# 1. Data

## 1.1 Data Acquisition  

<span style="color:red">
Provide a brief description of the exemplary data used in your use case.  
If necessary, include guidelines or tips for collecting custom datasets.  </span>


## 1.2. Data Anntation

<span style="color:red">
Include a step-by-step guide on how to annotate data such that it can be used in a plug and play fashion. This should allow EM researchers for example to exchange the segmentation of cellular structures with the segmentation of virus particles to match their labratories needs. 

For data annotation we recomment using the <a href="https://www.cvat.ai/">CVAT</a> (Computer Vision Annotation Tool) tool. For further instructions, we refer to our <a href="">Getting Started</a> page.

Example step-by-step guide is shown below for the case of annotating location annotations. </span>

### 1.2.1. Create a New Task

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-1.png" alt="Create New Task1" width="500">
  <br>
</p>
When starting CVAT, you first need to create a new task. You can give it a name, add annotation types and upload your data.

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-2.png" alt="Create New Task2" width="500">
  <br>
</p>

Next, click on the `Add label` button. Name it based on the class you want to annotate. In our case one of *"naked", "budding", "enveloped"*. As annotation type choose `Points`. You should also pick a color, as this will simplify the annotation process. For adding new class click `Continue`. Once you added all nessecary classes click `Cancel`. 


<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-3.png" alt="Create New Task3" width="500">
  <br>
</p>

Now you can upload the data you wish to annotate. Finally, click `Submit & Open` to continue with the annotation of the uploaded data. 

### 1.2.2. Annotation

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-4.png" alt="Annotate1" width="500">
  <br>
</p>

This will open following view. Click on the job (in this view the `Job #1150022`) to start the annotation job. 

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-5.png" alt="Annotate2" width="500">
  <br>
</p>

To then annotate your data, select the `Draw new points` tool. Select the Label you wish to annotate from the dropdown menue. Then click `Shape` to annotate individual virus capsids with the label class. (Track will allow you to place annotations over multiple frames, which is helpful when annotating videos, tomograms or similar). You can use the arrows on the top middle to navigate through all of your data and to see your annotation progress. 

### 1.2.3. Save Annotation

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-6.png" alt="Save1" width="500">
  <br>
</p>

Once you are done annotating data, click on the `Menu` and select `Export job dataset`. 

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-7.png" alt="Save2" width="500">
  <br>
</p>

During export select the `CVAT for Images 1.1` format and give the folder a name. It will prepare the dataset for download. If you have the annotated images stored locally, there is no need to enable `Save Images`. 

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-8.png" alt="Save3" width="500">
  <br>
</p>

In the horizontal menu bar at the top go to `Requests`. It will show a request Export Annotations. On the right of this request click on the three dots on the right to download your exported, annotated data. This will download a .zip file containing the annotation file in .xml format. The name of the file should be "annotations.xml".



## 1.3. Data Preprocessing

<span style="color:red">Include a step-by-step guide for preprocessing the data based on common EM data formats like `.tif`to match your notebooks needs.</span>

<span style="color:red">[ImageJ](https://imagej.net/ij/) can be a helpful software in this case, as it is an open source, Java-based image processing software that runs on multiple platforms and offers a wide range of features, including automation with macros, extensive community support, and a large library of tools and plugins.</span>

<span style="color:red">In the following we showcase an example usecase to import and export data into required formats. </span>





<p align="center">
  <img src="images/ImageJ-1.png" alt="ImageJ1" width="500">
  <br>
</p>

<span style="color:green">This tool allows to `import` a large amount of different, commonly used file formats im EM. </span>

<p align="center">
  <img src="images/ImageJ-2.png" alt="ImageJ2" width="500">
  <br>
</p>

<span style="color:green">Using the provided `Save As..` functionality allows to save the imported files as a `Image Sequence` in .tif format or single `.tif` files.</span>

## 1.4. Data Structuring

<span style="color:red">Your use case will require the data to be in a specific structure, such that it can be used for  training. Describe this structure here. </span>

<span style="color:green">**An example description could look like this:** </span>

<span style="color:green">The provided notebook requires that all training, validation and testing data is placed within a single folder. Splitting the data into train, test and validation will be done during runtime. </span>

<span style="color:green">Additionally, the generated `annotations.xml` should be put in the same folder as the .tif images.</span>

<span style="color:green">You can check the exemplary data provided at `data/tem-herpes/` for clarification.</span>

<span style="color:green">An example with five images and the corresponding annotation is shown below: </span>

```
/data/tem-herpes/
├── image_001.tif
├── image_002.tif
├── image_003.tif
├── image_004.tif
├── image_005.tif
└── annotations.xml

```

If you are using different data than the one provided (`data/`), we require you to set the path to this folder. To do so, adapt the path in the text form below.


In [ ]:
data_widget = create_text_widget("Data Path:","./data","Enter the path to your data folder.")
display(*data_widget)

In [ ]:
data_path = data_widget[0].value
print(f"[INFO]::Data path was set to: {data_path}")

# 2. Model Training

## 2.1. Setup Logging

By executing the cell below, we setup the logging directory for model training and evaluation. 
The logger creates a folder at `./logs/<datafoldername>-<currentdatetime>/`. 
Within this folder there will be logging of: 

- the used hyperparameters, (`<log-path>/TrainingRun/hyperparameters.json`)
- the best performing model checkpoint based on the validation loss (`<log-path>/TrainingRun/checkpoints/best_model.pth`)
- the last model checkpoint (`<log-path>/TrainingRun/checkpoints/latest_model.pth`)
- visualizations of training and validation curves (`<log-path>/TrainingRun/plots/training_curves.png`)
- qualitative visualization of sampled validation images (`<log-path>/TrainingRun/samples/`)
- results on test metrics (`<log-path>/TrainingRun/test_results.txt`)
- qualitative visualization of sampled test images (`<log-path>/TrainingRun/samples/`)

Additional text logs are being saved to `<log-path>/TrainingRun/log.txt` and `<log-path>/log.txt` 

<span style="color:red">Add a short description of what the sample visualizations show in your use case.</span>

In [ ]:
logger = Logger(data_path)

## 2.2. Hyperparameter Tuning


Hyperparameters in deep learning are configurable settings that define how a model is trained. Unlike model parameters, they are set before training and not learned from the data.

When training a model, we highly recommend to do a hyperparameter tuning first. By tuning the hyperparameters the model is usually trained on a subset of the data with a smaller number of epochs, and then evaluated based on its performance on the validation set. Then, hyperparameters, which lead to the best performance are chosen for full training of the model. 
Similar to the training run, all sweep runs will be logged. You can find the according logs at `<log-path>/Sweep-<idx>`.

Our workflow equips you, as EM experts, with an automatic hyperparameter search based on a grid search. The DL experts have chosen some basic setting for performing the hyperparameter seach and defining the search space. The DL experts also describe the individual hyperparameters. This allows you to further adapt the search space to your specific needs. 

In order to do so, you can adapt the form below. Each sweep parameter should be separated by `,`. Floating point values should be written like `0.1`. 

In [ ]:
# hyperparameter search
model_trainer = ModelTrainer(data_path, logger)

hyperparameter_tuner = ModelTuner(model_trainer, logger)
form = hyperparameter_tuner.create_hyperparameter_widgets()
display(form)


If you wish to run a hyperparameter sweep based on the parameters above, please execute the cell below.

In [ ]:
best_config = None
hyperparameter_tuner.update_config(form)
print("Sweep config:")
for k in hyperparameter_tuner.config['hyperparameter'].keys():
    print(f"\t{k}: {hyperparameter_tuner.config['hyperparameter'][k]['values']} (default: {hyperparameter_tuner.config['hyperparameter'][k]['default']})")
best_config = hyperparameter_tuner.tune()

Our automatic hyperparameter tuning is able to find the best performing set of hyperparameters based on the setting shown above. 

However, there can be scenarios, where additional flexibility is required. Therefore, you are able to change these hyperparameters in the following. 

**WARNING** This setting is for advanced users only. Please only change parameters here, if you know what you are doing. 

In [ ]:
form = hyperparameter_tuner.edit_hyperparameters()
display(form)

In [ ]:
best_config = hyperparameter_tuner.update_hyperparameters(form)
print_info(f"Will use following hyperparameters for future training: {best_config}")

## 2.3. Training and Validation

Training in deep learning is the process where a model learns patterns from labeled data by optimizing its parameters through backpropagation. 
Validation involves using a separate dataset to evaluate the model's performance during training, ensuring it generalizes well to unseen data.
Hence, in this section we train and validate the model based on the provided data and hyperparameters resulting from the previous sweep.

If no sweep was conducted (not recommended for new datasets!), the default parameters, defined by the DL expert will be used. 

In case the training run was cancelled, it can be resumed from a previous checkpoint. To do so, you need to provide a model checkpoint in the text form below. You can find these checkpoints inside the runs logging directory (`<log-dir>/TrainingRun/checkpoints/latest_model.pth`). If you do not wish to resume training, you can ignore the text form below.

In [ ]:
resume_widget = create_text_widget("Resume Training:","","If you wish to resume an earlier training, enter the path to the latest_model.pth file here.")
display(*resume_widget)

In [ ]:


resume_training = resume_widget[0].value
if(resume_training):
    resume_training = Path(resume_training)
    if(resume_training.is_dir()):
        resume_training = Path(find_file(resume_training, "latest_model.pth")) 
        # resume_training = Path(os.path.join(resume_training,"latest_model.pth"))
    if(not resume_training.is_file()):
        logger.log_error(f"Could not find resume path at {resume_training}. Will start training from scatch.")
        resume_training = None
    else: 
        logger.log_info(f"Will resume training from {resume_training}")
else:
    resume_training = None
logger.init("TrainingRun")
model_trainer.resume_from_checkpoint = resume_training
model_trainer.prepare(best_config)


If you wish to train a model, execute the cell below. 

In [ ]:
model_trainer.fit()

# 3. Model Evaluation
Evaluation in deep learning is the process of evaluating a trained model on a separate, unseen dataset to measure its final performance. It provides an unbiased assessment of the model's ability to generalize to new data.

## 3.1. Choose Model 

In this section we choose the model for testing. 
If you leave the `Model Path` empty in the text form below, it will use the last model trained.
Otherwise, you can define the path to the models best weights at `<log-path>/TrainingRun/checkpoints/best_model.pth` 

In [ ]:
model_widget = create_text_widget("Model Path:","","If you wish to test a specific model, you can here define the path to its checkpoint. (For example: logs/tem-herpes_2025-02-03_11-42-43/TrainingRun/checkpoints)")
display(*model_widget)

## 3.2. Evaluate
We finally evaluate the provided model on the test set. We investigate following metrics: 

<span style="color:red"> Add a description of the used metrics here. </span>

<span style="color:green">

- **Accuracy (Acc)**: The percentage of correct predictions based on all predictions.

- **Mean Absolute Error (MAE)**: A distance metric to compute the difference between the prediction and the label. Measures the absolute difference between predicted and actual counts. The lower the metric, the better. 

- **Mean Absolute Error (MAE-class-name)**: A distance metric to compute the difference between the prediction and the label. Measures the absolute difference between predicted and actual counts for each class individually. This can help to find a specifically well/bad performing class. The lower the metric, the better.
</span>


<span style="color:red">Add a description of your visualizations here.</span>




These visualizations are saved to `<log-path>/Evaluate/samples/test_*`. 

If you wish to evaluate a model, execute the cell below.

In [ ]:
from pathlib import Path 
start_evaluation = False
eval_model = model_widget[0].value
if(eval_model):
    eval_model = Path(eval_model)
    if(eval_model.is_dir()):
        eval_model = Path(find_file(eval_model, "best_model.pth")) 
    if(not eval_model.is_file()):
        logger.log_error(f"Could not find model at {eval_model}. Make sure to train a model before evaluation.")
        eval_model = None
    else: 
        start_evaluation = True
else:
    recent_logs = logger.get_most_recent_logs()
    eval_model = ""
    for dataname, log_path in recent_logs.items():
        if(dataname == Path(data_path).stem):
            eval_model = Path(log_path+"/TrainingRun/checkpoints/best_model.pth")
            if(not eval_model.is_file()):
                logger.log_error(f"Cound not find a trained model at {eval_model}. Make sure you fully train a model first before evaluating.")
            else:
                logger.log_info(f"Found most recent log at {eval_model}")
                start_evaluation = True
        else: 
            continue
    if(not start_evaluation):
        logger.log_error("Cound not find a trained model. Make sure you train a model first before evaluating.")
      
if(start_evaluation):
    model_trainer.load_checkpoint(eval_model)
    model_trainer.test()      
